# Mask R-CNN Demo

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

# Import the tabletop dataset config
import samples.humanoids_pouring.tabletop_bottles as tabletop

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
TRAINED_MODEL_PATH = os.path.join(MODEL_DIR, "bottles/bottles_exp_4", "mask_rcnn_bottles_0040.h5")
assert os.path.exists(TRAINED_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

## Configurations

In [ ]:
config = tabletop.configurations.YCBVideoConfigInference()
config.DETECTION_MIN_CONFIDENCE = 0.8
config.display()

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = config.GPU_ID

## Create model and load trained weights

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights
model.load_weights(TRAINED_MODEL_PATH, by_name=True)

## Load Validation dataset and classes

In [ ]:
dataset_root_dir = os.path.join(ROOT_DIR, "datasets", "bottles")
dataset_val = tabletop.datasets.YCBVideoDataset()
dataset_val.load_dataset(dataset_root_dir, 'val')

## Load image to run detection on

In [ ]:
# Load a random image from val set
random_val_img_info = dataset_val.image_info[random.randint(0, len(dataset_val.image_info))]
image = skimage.io.imread(random_val_img_info['path'])

In [ ]:
# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

## Run object detection

In [ ]:
# Run detection
import time
start_time = time.time()

results = model.detect([image], verbose=1)

print("--- %s seconds ---" % (time.time() - start_time))

# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'])